Author: Guy, Michael, Joshua
Summary: This is a comprenhensive simulation engine for ambulance simulation. It uses the solution from two-stage stochastic program and robust programs to simulate how ambulances would perform. The simulation mostly runs using a priority-queue of events with respect to time. Events include things like ambulance arrives to the scene of call, ambulance arrives to hospital, ect.

In [1]:
import Pkg #this line doesn't seem to run either way for me
Pkg.add("Pkg")
Pkg.add("Dates")

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [2]:
"""This activates packages, namely ambulance deployment"""

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`


Warning: Package AmbulanceDeployment does not have Random in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AmbulanceDeployment
└ Loading Random into AmbulanceDeployment from project dependency, future warnings for AmbulanceDeployment are suppressed.

In [3]:
using AmbulanceDeployment

In [4]:
using DataFrames, JLD, Distributions, CSV, Random, JSON, DelimitedFiles, Dates #Plots I don't think we use Plots
import DataStructures: PriorityQueue, enqueue!, dequeue!
include(".//src//problem.jl")
include(".//src//model.jl")

In [5]:
"""Helper functions for the simuation engine?"""
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 1 method)

In [82]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)#Distributions.LogNormal(3.089, 0.3) #Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Robust01"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

num_regions = 3199
hospitals = CSV.File(string(local_path,"../test/austin_data_3200/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin_data_3200/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin_data_3200/WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin_data_3200/adjacent_nbhd.csv") |> DataFrame
test_calls = CSV.File("../test/austin_data_3200/austin_test_calls_v4.csv")|> DataFrame 
coverage = CSV.read("../test/austin_data_3200/coverage_regression.csv", DataFrame, header=false) #coverage_real is austin-data
coverage = delete!(coverage,1) #trim df header (1st row)
coverage = select(coverage,2:3234) #trim df header (1st column)
coverage = Matrix(coverage)
#coverage = convert(Array{Bool, 2}, coverage)
coverage = convert(Array{Bool,2},coverage')

# num_regions = 200
# hospitals = CSV.File(string(local_path,"../test/austin_data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin_data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin_data/WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin_data/adjacent_nbhd.csv") |> DataFrame
#CALLS IN MINUTES #test_calls = CSV.File("../test/austin_data/austin_test_calls_v1.csv")|> DataFrame  
# coverage = CSV.read("../test/austin_data/coverage.csv", DataFrame, header=false) #coverage_real is austin-data
# coverage = Matrix(coverage)
# coverage = convert(Array{Bool, 2}, coverage)

solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


include("..//src//problem.jl")

p = DeploymentProblem(
          num_regions,
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )

print("n ambulances ",p.nambulances)
print("\n locations  ",p.nlocations)
print("\n nregions ",p.nregions)
# print("\n train ", p.train)
# print("\n test ", p.test)
print("\n coverage ",size(p.coverage))
print("\n demand ",size(p.demand))
print("\n adjacency ",size(p.adjacency))

#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]#convert(Array{Int,1},ones(44))#amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)
Random.seed!(1234)

n ambulances 40
 locations  44
 nregions 3199
 coverage (3233, 44)
 demand (0, 2)
 adjacency (3199, 3199)running 40 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

###### problem.available

In [70]:
function logRegression(gridTime)
    if(gridTime < 0)
        print("grid time less than 0: ", gridTime)
    end
    gridTime = gridTime / 60
#         slope = 0.23868463365149437
#         intercept = 1.261777659720721
    slope= 0.4011478655815724 #test calls
    intercept= 1.0469242277715907 #test calls
    ax = log(gridTime)*slope
    new_time = exp(ax + intercept)
    new_time =  new_time * 60
    new_time
end

logRegression (generic function with 1 method)

In [71]:
"""Simulation Engine"""

Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        
        travel_time = ceil(Int, problem.emergency_calls[id, Symbol("stn$(i)_min")]) #60*
        println(travel_time)
        travel_time = logRegression(travel_time) # gridTime2Bin(travel_time) #adjust time
        travel_time = ceil(travel_time) #time mod
        println(travel_time)
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
        println("and there are this many amb available: ", sum(problem.available))
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    #60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes why the hell would we add 15 minutes and then also the hopsital min!! smh
    conveytime = ceil(Int, problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    conveytime = ceil(logRegression(conveytime)/60) # minutes #adjust time
    #conveytime = 15
    ems.eventlog[id, :conveytime] = conveytime 
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    returntime = ceil(logRegression(returntime)) #adjust time
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [83]:
"""call simulation"""
#@time df, guiArray = simulate_events!(problem, dispatch)
df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 1012
closest available station : 13
327
338.0
Ambulance has arrived on the scene for event 1 at time 376
calling event id: 2 time: 575 value: 220
closest available station : 31
646
444.0
calling event id: 3 time: 673 value: 1687
closest available station : 25
477
393.0
calling event id: 4 time: 691 value: 1536
closest available station : 33
328
338.0
Ambulance has arrived on the scene for event 2 at time 1019
Ambulance has arrived on the scene for event 4 at time 1029
Ambulance has arrived on the scene for event 3 at time 1066
calling event id: 5 time: 1166 value: 1621
closest available station : 25
465
389.0
calling event id: 6 time: 1187 value: 578
closest available station : 2
613
435.0
calling event id: 7 time: 1450 value: 1169
closest available station : 16
927
513.0
Ambulance has arrived on the scene for event 5 at time 1555
Ambulance for event 1 has arrived at the hospital at time 1574
calling event id: 8 time: 1578 value: 825
closest availabl

Ambulance for event 26 is returning to the station at 8170
Ambulance has arrived on the scene for event 27 at time 8229
Ambulance has arrived on the scene for event 29 at time 8371
Ambulance has arrived on the scene for event 30 at time 8397
Ambulance has arrived on the scene for event 28 at time 8443
calling event id: 31 time: 8588 value: 760
closest available station : 18
535
412.0
Ambulance from event 26 has returned to station 21
Ambulance from event 23 has returned to station 21
calling event id: 32 time: 8673 value: 1008
closest available station : 18
856
497.0
Ambulance for event 25 has arrived at the hospital at time 8761
Ambulance for event 25 is returning to the station at 8768
Ambulance has arrived on the scene for event 31 at time 9000
calling event id: 33 time: 9154 value: 575
closest available station : 2
342
344.0
Ambulance has arrived on the scene for event 32 at time 9170
Ambulance for event 27 has arrived at the hospital at time 9196
Ambulance from event 25 has return

closest available station : 25
377
358.0
calling event id: 55 time: 15714 value: 1437
closest available station : 21
409
370.0
calling event id: 56 time: 15831 value: 1664
closest available station : 34
475
392.0
Ambulance has arrived on the scene for event 53 at time 15910
Ambulance for event 52 has arrived at the hospital at time 15949
Ambulance for event 52 is returning to the station at 15955
Ambulance from event 50 has returned to station 25
Ambulance has arrived on the scene for event 54 at time 16057
Ambulance has arrived on the scene for event 55 at time 16084
Ambulance from event 51 has returned to station 44
Ambulance has arrived on the scene for event 56 at time 16223
calling event id: 57 time: 16247 value: 1009
closest available station : 21
829
491.0
Ambulance from event 52 has returned to station 13
Ambulance for event 53 has arrived at the hospital at time 16662
Ambulance for event 53 is returning to the station at 16667
Ambulance has arrived on the scene for event 57 at

Ambulance for event 76 has arrived at the hospital at time 25851
Ambulance for event 76 is returning to the station at 25858
calling event id: 79 time: 25937 value: 702
closest available station : 13
672
451.0
Ambulance from event 77 has returned to station 21
Ambulance from event 78 has returned to station 21
Ambulance from event 76 has returned to station 25
Ambulance has arrived on the scene for event 79 at time 26388
calling event id: 80 time: 26570 value: 1625
closest available station : 25
463
388.0
calling event id: 81 time: 26618 value: 1257
closest available station : 21
395
365.0
calling event id: 82 time: 26633 value: 1568
closest available station : 42
230
294.0
calling event id: 83 time: 26778 value: 1380
closest available station : 4
254
305.0
Ambulance has arrived on the scene for event 82 at time 26927
Ambulance has arrived on the scene for event 80 at time 26958
Ambulance has arrived on the scene for event 81 at time 26983
Ambulance has arrived on the scene for event 8

Ambulance has arrived on the scene for event 104 at time 38310
calling event id: 106 time: 38341 value: 458
closest available station : 31
692
456.0
Ambulance has arrived on the scene for event 105 at time 38498
calling event id: 107 time: 38642 value: 1865
closest available station : 17
407
369.0
Ambulance for event 103 has arrived at the hospital at time 38737
calling event id: 108 time: 38739 value: 1624
closest available station : 25
357
350.0
Ambulance for event 103 is returning to the station at 38743
Ambulance has arrived on the scene for event 106 at time 38797
Ambulance has arrived on the scene for event 107 at time 39011
Ambulance for event 104 has arrived at the hospital at time 39013
Ambulance for event 104 is returning to the station at 39018
Ambulance from event 103 has returned to station 13
calling event id: 109 time: 39060 value: 1808
closest available station : 25
349
347.0
Ambulance has arrived on the scene for event 108 at time 39089
Ambulance has arrived on the sce

...

Ambulance from event 906 has returned to station 18
calling event id: 910 time: 255367 value: 1619
closest available station : 17
587
427.0
calling event id: 911 time: 255429 value: 1379
closest available station : 21
262
309.0
Ambulance for event 907 has arrived at the hospital at time 255440
Ambulance for event 907 is returning to the station at 255446
calling event id: 912 time: 255506 value: 1440
closest available station : 21
301
327.0
calling event id: 913 time: 255613 value: 886
closest available station : 13
512
404.0
calling event id: 914 time: 255645 value: 890
closest available station : 21
1029
535.0
Ambulance has arrived on the scene for event 911 at time 255738
Ambulance for event 909 has arrived at the hospital at time 255772
Ambulance for event 909 is returning to the station at 255782
Ambulance has arrived on the scene for event 910 at time 255794
Ambulance has arrived on the scene for event 912 at time 255833
Ambulance from event 907 has returned to station 18
Ambulan

611
434.0
Ambulance for event 928 has arrived at the hospital at time 261221
Ambulance has arrived on the scene for event 937 at time 261221
Ambulance for event 928 is returning to the station at 261231
Ambulance has arrived on the scene for event 936 at time 261246
Ambulance for event 933 has arrived at the hospital at time 261270
Ambulance for event 930 has arrived at the hospital at time 261270
Ambulance for event 933 is returning to the station at 261278
Ambulance for event 930 is returning to the station at 261280
Ambulance from event 929 has returned to station 26
Ambulance from event 927 has returned to station 21
Ambulance for event 932 has arrived at the hospital at time 261335
Ambulance for event 932 is returning to the station at 261341
Ambulance has arrived on the scene for event 938 at time 261445
calling event id: 940 time: 261500 value: 1315
closest available station : 21
480
394.0
calling event id: 941 time: 261510 value: 1562
closest available station : 25
643
443.0
ca

279.0
Ambulance from event 954 has returned to station 2
calling event id: 963 time: 266066 value: 2225
closest available station : 41
426
376.0
Ambulance has arrived on the scene for event 961 at time 266155
Ambulance has arrived on the scene for event 959 at time 266174
Ambulance has arrived on the scene for event 960 at time 266277
Ambulance has arrived on the scene for event 962 at time 266318
Ambulance from event 953 has returned to station 25
Ambulance from event 955 has returned to station 2
calling event id: 964 time: 266425 value: 1568
closest available station : 42
230
294.0
Ambulance has arrived on the scene for event 963 at time 266442
Ambulance for event 958 has arrived at the hospital at time 266683
Ambulance for event 958 is returning to the station at 266691
calling event id: 965 time: 266716 value: 642
closest available station : 27
852
496.0
Ambulance has arrived on the scene for event 964 at time 266719
Ambulance for event 956 has arrived at the hospital at time 2668

322
336.0
Ambulance for event 981 has arrived at the hospital at time 275916
Ambulance for event 981 is returning to the station at 275923
calling event id: 985 time: 276131 value: 1314
closest available station : 21
696
457.0
Ambulance for event 983 has arrived at the hospital at time 276192
Ambulance for event 983 is returning to the station at 276198
Ambulance has arrived on the scene for event 984 at time 276202
Ambulance from event 981 has returned to station 21
Ambulance for event 982 has arrived at the hospital at time 276469
Ambulance for event 982 is returning to the station at 276477
Ambulance has arrived on the scene for event 985 at time 276588
Ambulance from event 983 has returned to station 4
calling event id: 986 time: 276780 value: 1684
closest available station : 25
203
279.0
Ambulance for event 984 has arrived at the hospital at time 276937
Ambulance for event 984 is returning to the station at 276942
Ambulance from event 982 has returned to station 2
Ambulance has ar

(1000×11 DataFrame
  Row │ id     dispatch_from  waittime  responsetime  scenetime  conveytime  r ⋯
      │ Int64  Int64          Float64   Float64       Float64    Float64     F ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │     1             13       0.0       5.63333    19.9667         7.0    ⋯
    2 │     2             31       0.0       7.4        11.75          10.0
    3 │     3             25       0.0       6.55       11.75           9.0
    4 │     4             33       0.0       5.63333    13.2833         9.0
    5 │     5             25       0.0       6.48333    19.95           7.0    ⋯
    6 │     6              2       0.0       7.25       29.8833         8.0
    7 │     7             16       0.0       8.55       18.0667         9.0
    8 │     8             18       0.0       7.5        14.2            5.0
    9 │     9             27       0.0       8.15       17.9            7.0    ⋯
   10 │    10             13       0.0 

In [84]:
println(mean(df[!,"responsetime"]))
println(mean(df[!,"scenetime"]))
println(mean(df[!,"conveytime"]))
println(mean(df[!,"returntime"]))

6.690766666666667
16.000433333333337
7.106
7.268400000000001


## Temp results journal 

println(mean(df["responsetime"]))
println(mean(df["scenetime"]))
println(mean(df["conveytime"]))
println(mean(df["returntime"]))
println(mean(df[!,"returntime"]))
#### lognromal 3.08
9.813166666666667 <br>
9.134116666666667 <br>
24.4176 <br>
8.908 <br>
#### lognromal 3.65
10.3443 <br>
16.000433333333337 <br>
24.4176 <br>
9.090300000000001 <br>
#### adjust travel time by time_bin weights
10.466350000000002 <br>
16.000433333333334 <br>
24.4176 <br>
9.073 <br>
#### global
7.7477 <br>
16.250616666666666 <br>
24.4176 <br>
9.114916666666668 <br>
#### log-log 3200 grid UNIFORM 44 WHOOPS
5.3978 <br>
16.000433333333337 <br>
15.4032 <br>
8.835816666666668 <br>
#### 3200 grid UNIFORM 44  WHOOPS
6.387766666666668 <br>
16.000433333333334 <br>
15.4032 <br>
8.805600000000002 <br>
#### 3200 grid stochastic
8.145316666666668
16.25061666666667
15.4032
9.455366666666668
#### old coefficients log-log 3200 grid stochastic 
5.700449999999999
16.385216666666665
6.282
5.874950000000001
#### new coefficients log-log 3200 grid stochastic
6.420166666666667
16.3654
7.106
6.756500000000001
#### v4 test calls ->
6.153766666666668
16.000433333333334
7.106
6.721716666666667
#### convey time looks too high try to change it to a static 15 minutes -> there was a /60 error again



both coefficients work.. which to use.

In [ ]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)#Distributions.LogNormal(3.089, 0.3) #Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Robust01"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

num_regions = 3199
hospitals = CSV.File(string(local_path,"../test/austin_data_3200/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin_data_3200/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin_data_3200/WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin_data_3200/adjacent_nbhd.csv") |> DataFrame
test_calls = CSV.File("../test/austin_data_3200/austin_test_calls_v4.csv")|> DataFrame 
coverage = CSV.read("../test/austin_data_3200/coverage_regression.csv", DataFrame, header=false) #coverage_real is austin-data
coverage = delete!(coverage,1) #trim df header (1st row)
coverage = select(coverage,2:3234) #trim df header (1st column)
coverage = Matrix(coverage)
#coverage = convert(Array{Bool, 2}, coverage)
coverage = convert(Array{Bool,2},coverage')

# num_regions = 200
# hospitals = CSV.File(string(local_path,"../test/austin_data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin_data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin_data/WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin_data/adjacent_nbhd.csv") |> DataFrame
#CALLS IN MINUTES #test_calls = CSV.File("../test/austin_data/austin_test_calls_v1.csv")|> DataFrame  
# coverage = CSV.read("../test/austin_data/coverage.csv", DataFrame, header=false) #coverage_real is austin-data
# coverage = Matrix(coverage)
# coverage = convert(Array{Bool, 2}, coverage)

solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


include("..//src//problem.jl")

p = DeploymentProblem(
          num_regions,
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )

print("n ambulances ",p.nambulances)
print("\n locations  ",p.nlocations)
print("\n nregions ",p.nregions)
# print("\n train ", p.train)
# print("\n test ", p.test)
print("\n coverage ",size(p.coverage))
print("\n demand ",size(p.demand))
print("\n adjacency ",size(p.adjacency))

#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]#convert(Array{Int,1},ones(44))#amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)
Random.seed!(1234)

In [88]:
"""batch call simulation"""
batches = 12
test_calls = CSV.File("../test/austin_data_3200/austin_test_calls_v4.csv")|> DataFrame 
include("..//src//problem.jl")
model_name = "Robust01"

for i = 1:12
    """ simulation parameters """
    p = DeploymentProblem(
          num_regions,
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )

    #testcalls
    start_ind = (i-1)*1000+1
    end_ind = (i)*1000
    println("running $namb ambuances & $start_ind to $end_ind calls")

    test_calls_batch = test_calls[start_ind:end_ind,:]

    # distribution of ambulances across all of the stations
    x = amb_deployment[model_dict[model_name]][namb]

    #establishes the dispatch problem
    problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)

    # #establishes the dispatch model
    dispatch = ClosestDispatch(p, problem)

    df, guiArray = simulate_events!(problem, dispatch)
    print(mean(df[!,"responsetime"]))

    filename = string(model_name,string(namb),"calls",string(ncalls),"batch",string(i),".csv")
    open(filename, "w") do filename
        writedlm(filename,  df[!,"responsetime"])
    end
    println(filename)
end

running 40 ambuances & 1 to 1000 calls
calling event id: 1 time: 38 value: 1012
closest available station : 13
327
338.0
Ambulance has arrived on the scene for event 1 at time 376
calling event id: 2 time: 575 value: 220
closest available station : 31
646
444.0
calling event id: 3 time: 673 value: 1687
closest available station : 25
477
393.0
calling event id: 4 time: 691 value: 1536
closest available station : 33
328
338.0
Ambulance has arrived on the scene for event 2 at time 1019
Ambulance has arrived on the scene for event 4 at time 1029
Ambulance for event 1 has arrived at the hospital at time 1059
Ambulance has arrived on the scene for event 3 at time 1066
Ambulance for event 1 is returning to the station at 1066
calling event id: 5 time: 1166 value: 1621
closest available station : 25
465
389.0
calling event id: 6 time: 1187 value: 578
closest available station : 2
613
435.0
Ambulance from event 1 has returned to station 13
calling event id: 7 time: 1450 value: 1169
closest avai

Ambulance has arrived on the scene for event 29 at time 8371
Ambulance has arrived on the scene for event 30 at time 8397
Ambulance has arrived on the scene for event 28 at time 8412
Ambulance from event 23 has returned to station 21
calling event id: 31 time: 8588 value: 760
closest available station : 18
535
412.0
calling event id: 32 time: 8673 value: 1008
closest available station : 18
856
497.0
Ambulance for event 25 has arrived at the hospital at time 8797
Ambulance for event 25 is returning to the station at 8804
Ambulance for event 26 has arrived at the hospital at time 8947
Ambulance for event 26 is returning to the station at 8954
Ambulance has arrived on the scene for event 31 at time 9000
calling event id: 33 time: 9154 value: 575
closest available station : 2
342
344.0
Ambulance has arrived on the scene for event 32 at time 9170
Ambulance for event 29 has arrived at the hospital at time 9197
Ambulance for event 29 is returning to the station at 9206
Ambulance from event 25

Ambulance for event 51 has arrived at the hospital at time 16175
Ambulance for event 51 is returning to the station at 16183
Ambulance for event 50 has arrived at the hospital at time 16208
Ambulance for event 50 is returning to the station at 16215
Ambulance has arrived on the scene for event 56 at time 16223
calling event id: 57 time: 16247 value: 1009
closest available station : 21
829
491.0
Ambulance from event 49 has returned to station 42
Ambulance from event 52 has returned to station 13
Ambulance for event 53 has arrived at the hospital at time 16724
Ambulance for event 53 is returning to the station at 16729
Ambulance from event 50 has returned to station 25
Ambulance has arrived on the scene for event 57 at time 16738
Ambulance from event 51 has returned to station 44
Ambulance for event 56 has arrived at the hospital at time 16853
Ambulance for event 56 is returning to the station at 16860
Ambulance for event 54 has arrived at the hospital at time 17062
Ambulance for event 5

463
388.0
Ambulance from event 76 has returned to station 25
Ambulance from event 78 has returned to station 21
calling event id: 81 time: 26618 value: 1257
closest available station : 21
395
365.0
calling event id: 82 time: 26633 value: 1568
closest available station : 42
230
294.0
calling event id: 83 time: 26778 value: 1380
closest available station : 4
254
305.0
Ambulance has arrived on the scene for event 82 at time 26927
Ambulance has arrived on the scene for event 80 at time 26958
Ambulance has arrived on the scene for event 81 at time 26983
Ambulance has arrived on the scene for event 83 at time 27083
Ambulance for event 79 has arrived at the hospital at time 27242
Ambulance for event 79 is returning to the station at 27248
calling event id: 84 time: 27471 value: 513
closest available station : 2
93
204.0
Ambulance for event 80 has arrived at the hospital at time 27659
Ambulance for event 80 is returning to the station at 27668
Ambulance for event 82 has arrived at the hospital

369.0
calling event id: 108 time: 38739 value: 1624
closest available station : 25
357
350.0
Ambulance has arrived on the scene for event 106 at time 38797
Ambulance from event 103 has returned to station 13
Ambulance has arrived on the scene for event 107 at time 39011
calling event id: 109 time: 39060 value: 1808
closest available station : 25
349
347.0
Ambulance has arrived on the scene for event 108 at time 39089
Ambulance for event 105 has arrived at the hospital at time 39236
Ambulance for event 105 is returning to the station at 39243
Ambulance has arrived on the scene for event 109 at time 39407
Ambulance from event 105 has returned to station 25
Ambulance for event 104 has arrived at the hospital at time 39680
Ambulance for event 104 is returning to the station at 39685
Ambulance for event 106 has arrived at the hospital at time 39687
Ambulance for event 106 is returning to the station at 39695
Ambulance for event 107 has arrived at the hospital at time 39852
Ambulance for eve

...

337
342.0
Ambulance for event 517 has arrived at the hospital at time 136825
Ambulance for event 517 is returning to the station at 136831
calling event id: 525 time: 136878 value: 1070
closest available station : 21
707
460.0
Ambulance has arrived on the scene for event 521 at time 136907
calling event id: 526 time: 136955 value: 1190
closest available station : 18
942
516.0
Ambulance has arrived on the scene for event 522 at time 137079
Ambulance for event 518 has arrived at the hospital at time 137102
Ambulance for event 518 is returning to the station at 137108
Ambulance has arrived on the scene for event 524 at time 137112
calling event id: 527 time: 137168 value: 1194
closest available station : 25
789
481.0
Ambulance has arrived on the scene for event 523 at time 137178
Ambulance from event 517 has returned to station 18
calling event id: 528 time: 137199 value: 1315
closest available station : 17
860
498.0
Ambulance for event 519 has arrived at the hospital at time 137313
Ambul

closest available station : 25
357
350.0
Ambulance for event 542 is returning to the station at 143018
Ambulance has arrived on the scene for event 544 at time 143110
calling event id: 547 time: 143126 value: 1496
closest available station : 17
767
476.0
Ambulance has arrived on the scene for event 545 at time 143127
Ambulance from event 543 has returned to station 13
Ambulance has arrived on the scene for event 546 at time 143366
Ambulance from event 542 has returned to station 21
Ambulance has arrived on the scene for event 547 at time 143602
Ambulance for event 545 has arrived at the hospital at time 143745
Ambulance for event 545 is returning to the station at 143754
calling event id: 548 time: 144012 value: 642
closest available station : 13
588
428.0
calling event id: 549 time: 144036 value: 405
closest available station : 43
531
410.0
calling event id: 550 time: 144202 value: 1989
closest available station : 25
539
413.0
Ambulance from event 545 has returned to station 27
callin

Ambulance has arrived on the scene for event 570 at time 148427
Ambulance from event 565 has returned to station 43
Ambulance has arrived on the scene for event 571 at time 148545
Ambulance for event 569 has arrived at the hospital at time 148632
Ambulance for event 569 is returning to the station at 148637
Ambulance has arrived on the scene for event 572 at time 148676
Ambulance for event 567 has arrived at the hospital at time 148692
Ambulance for event 567 is returning to the station at 148700
calling event id: 573 time: 148722 value: 830
closest available station : 13
787
480.0
Ambulance from event 566 has returned to station 18
Ambulance from event 568 has returned to station 21
calling event id: 574 time: 148782 value: 1441
closest available station : 21
236
297.0
Ambulance from event 569 has returned to station 21
Ambulance has arrived on the scene for event 574 at time 149079
Ambulance from event 567 has returned to station 25
calling event id: 575 time: 149179 value: 1562
clos

Excessive output truncated after 524348 bytes.

Ambulance has arrived on the scene for event 590 at time 155595
Ambulance for event 588 has arrived at the hospital at time 155598
Ambulance for event 587 has arrived at the hospital at time 155605


In [55]:
"""batch call simulation"""
# batches = 12
# test_calls = CSV.File("../test/austin_data_3200/austin_test_calls_v4.csv")|> DataFrame 
# #update 1: remove negative indices
# using DelimitedFiles

# for i = 1:12
#     """ simulation parameters """
#     p = DeploymentProblem(
#               hourly_calls,
#               adjacent_nbhd,
#               coverage,
#               namb = namb,
#               train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#               )
#     #testcalls
#     start_ind = (i-1)*1000+1
#     end_ind = (i)*1000
#     println("running $namb ambuances & $start_ind to $end_ind calls")
#     test_calls_batch = test_calls[start_ind:end_ind,:]

#     # distribution of ambulances across all of the stations
#     x = amb_deployment[model_dict[model_name]][namb]

#     #establishes the dispatch problem
#     problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)

#     #establishes the dispatch model
#     dispatch = ClosestDispatch(p, problem)

#     #@time df, guiArray = simulate_events!(problem, dispatch)
#     #df, guiArray = simulate_events!(problem, dispatch)
#     #print(mean(df["responsetime"]))
    
#     filename = string(model_name,string(namb),"calls",string(ncalls),"batch",string(i),".csv")
#     open(filename, "w") do filename
#         writedlm(filename,  df["responsetime"])
#     end
#     print(filename)
#end

"batch call simulation"

In [28]:
"""all models simulation"""
# turnaround = Distributions.LogNormal(3.65, 0.3)
# lambda = 0
# local_path = ""
# model_name = "Stochastic"
# namb = 40
# ncalls = 1000
# #using AmbulanceDeployment

# pwd()

# cd(currentpath * "/test/")
# pwd()

# #using AmbulanceDeployment
# using Dates

# #include("..//src//dispatch/closestdispatch.jl")
# include("..//src//problem.jl")
# #include("..//src//evaluate.jl") #if you include, you need to include JuMP
# #include("..//src//simulate.jl")

# hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
# coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
# coverage = convert(Array{Bool, 2}, coverage[:, :])
# incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
# solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
# amb_deployment = solverstats["amb_deployment"]
# model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
#           "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


# p = DeploymentProblem(
#           hourly_calls,
#           adjacent_nbhd,
#           coverage,
#           namb = namb,
#           train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#           )
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# test_calls = test_calls[1:ncalls,:]
# println("running $namb ambuances & $ncalls calls")

# # distribution of ambulances across all of the stations
# x = amb_deployment[model_dict[model_name]][namb]

# #establishes the dispatch problem
# problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

# #establishes the dispatch model
# dispatch = ClosestDispatch(p, problem)

# Random.seed!(1234)

"all models simulation"

In [29]:
array = filter(x->typeof(x)==gui_event,guiArray)
responded_array = filter(x->x.event_type == "call responded", array)

1000-element Array{Any,1}:
 gui_event("call responded", 1012, 1, 1, 1, -1, 0, DateTime("2021-07-19T12:39:09.102"))
 gui_event("call responded", 220, 31, 2, 31, -1, 0, DateTime("2021-07-19T12:39:09.196"))
 gui_event("call responded", 1687, 36, 3, 36, -1, 0, DateTime("2021-07-19T12:39:09.258"))
 gui_event("call responded", 1536, 33, 4, 33, -1, 0, DateTime("2021-07-19T12:39:09.304"))
 gui_event("call responded", 1621, 14, 5, 14, -1, 0, DateTime("2021-07-19T12:39:09.397"))
 gui_event("call responded", 578, 22, 6, 22, -1, 0, DateTime("2021-07-19T12:39:09.49"))
 gui_event("call responded", 1169, 16, 7, 16, -1, 0, DateTime("2021-07-19T12:39:09.505"))
 gui_event("call responded", 825, 8, 8, 8, -1, 0, DateTime("2021-07-19T12:39:09.721"))
 gui_event("call responded", 641, 9, 9, 9, -1, 0, DateTime("2021-07-19T12:39:09.8"))
 gui_event("call responded", 825, 3, 10, 3, -1, 0, DateTime("2021-07-19T12:39:09.892"))
 gui_event("call responded", 765, 35, 11, 35, -1, 0, DateTime("2021-07-19T12:39:09.985")

In [69]:
using DelimitedFiles
filename = string("visualization/time_adjust/",model_name,string(namb),"calls",string(ncalls),"_3200grid_regression2",".csv")
open(filename, "w") do filename
    writedlm(filename,  df[!,"responsetime"])
end
print(filename)

visualization/time_adjust/Stochastic40calls1000_3200grid_regression2.csv

In [48]:
#using CSV
#filename = string(joshpath,"test/",model_name,string(namb),"calls",string(ncalls),".csv")
#CSV.write(filename, df, delim=",")